# YoloV5 Training at GPU -- Applying Gaussian Filter at Ignored Areas

Treinamento de uma rede YOLOv5-L nos dados do UAVDT Dataset. As imagens com zona de "ignore" são tratadas; nesses boxes, é aplicado um filtro gaussiano para "borrar" a imagem de forma a não deixar nenhum contorno aparente.

Autor: Yuri Gonçalves Ribeiro

Data: 08 de Outubro de 2020

Exemplo de Execução:

```bash
conda activate yv5; experimento="YOLOv5_UAVDT_1_TRAIN"; outputFileName=$(date "+"$experimento"_ipynb_%d_%B_%Y_%Hh_%Mm_%Ss"); echo "Salvando log de execucao em: $outputFileName.txt"; nohup jupyter nbconvert --to notebook --execute --allow-errors --output "./logs/$outputFileName" --ExecutePreprocessor.timeout=-1 --Application.log_level=10 $experimento.ipynb &> "./logs/$outputFileName.txt" &
```

Os resultados são salvos na pasta do Dropbox especificada abaixo.

# Etiqueta do Experimento (Yolov5 Commit Hash, Conda Env, Informações da Máquina)

## Commit Hash da Rede YoloV5 utilizada neste experimento e pacotes instalados no conda environment

In [ ]:
%%script bash
echo "Hash key do commit YoloV5 que esta em uso:"
cd ../../Submodules/yolov5
echo $( git rev-parse --verify HEAD)
echo
echo "Verificar localizacao do python sendo executado:"
which python
echo
echo "Conda Environment Utilizado: "
conda env export

## Especificações do Computador e do Experimento

In [ ]:
# Especificações do computador, para saber onde estou rodando o código...
import datetime
from platform import uname
from os import environ
from glob import glob
import cv2

start_time = datetime.datetime.now()
time_stamp = start_time.strftime("%d_%B_%Y_%Hh_%Mm_%Ss")

username = environ['USER']
sysinfo = uname()
hostname = sysinfo.node

root = "~/Desktop/Mestrado/Codigos_Dissertacao/RedesNeurais/YOLOv5_UAVDT_1"

dropbox_dir = "~/Dropbox"

experimento = "YOLOv5_UAVDT_1"

output_dir = "%s/YoloV5_Runs/Experimentos/%s_%s"%(dropbox_dir,experimento,time_stamp)


# Checar conda environment
conda_env = !printenv CONDA_PROMPT_MODIFIER
if conda_env != ['(yv5) ']:
    raise SystemExit("ERRO.. Conda environment incorreto.")

In [ ]:
print("""
==Computer Specifications==
username: {username}

hostname: {hostname}

system info: {sysinfo}

{{root}}: {root}

dropbox dir: {dropbox_dir}

experimento: {experimento}

time stamp: {time_stamp}

output dir: {output_dir}
===========================
""".format(username = username, hostname = hostname, sysinfo = sysinfo,
           conda_env = conda_env, root = root, dropbox_dir = dropbox_dir,
           time_stamp = time_stamp, experimento = experimento, output_dir = output_dir))

In [ ]:
!nvidia-smi

# Clonar o Repositório YOLOv5 e instalar os requirements.txt

In [ ]:
#Ir para a raiz
%cd {root}
!echo
# Remover diretórios antigos
!rm -rf runs yolov5
#Criar pasta de runs
!mkdir runs
!echo
#Copiar diretório novo da pasta submodules
!cp -av ../../Submodules/yolov5/ yolov5/
!echo
!ls
#Entrar na pasta yolov5 e instalar os requirements.txt
%cd {root}/yolov5/
%pip install -U -r requirements.txt

# Executar Script para Transformar Imagens do UAVDT Dataset para o formato requerido pelo YOLOv5

In [ ]:
# Ir para a raiz
%cd {root}

In [ ]:
%%script bash
# Apagar os links existentes para o dataset
rm -rfv UAVDT_YOLOv5
# Criar novos hard links para o dataset e organizer o dataset no formato exigido pelo yolov5
mkdir UAVDT_YOLOv5
# Executar o script para converter UAVDT para o formato YOLOv5:
cd ../../misc

python uavdt_to_yolov5.py --conjunto treinoEteste --dest ../RedesNeurais/YOLOv5_UAVDT_1/UAVDT_YOLOv5 --nosymlink

cd ../RedesNeurais/YOLOv5_UAVDT_1/

# Preprocessar as imagens
## Salvá-las temporariamente (enquanto estiver treinando a rede). Após o término do treinamento, deve-se apagar as imagens objetivando economizar espaço em disco.

In [ ]:
movies_treino = ["M0101", "M0201", "M0202", "M0204", \
    "M0206", "M0207", "M0210", "M0301", "M0401", "M0402", \
    "M0501", "M0603","M0604", "M0605", "M0702", "M0703", \
    "M0704", "M0901", "M0902", "M1002", "M1003", "M1005", \
    "M1006", "M1008", "M1102", "M1201", "M1202", "M1304", \
    "M1305", "M1306"]

for movie_name in movies_treino:
    print(f"[INFO] Analisando a sequência de imagens {movie_name}.")

    frame_boxes = {}
    with open(f"../../Datasets/UAVDT/UAV-benchmark-MOTD_v1.0/GT/{movie_name}_gt_ignore.txt") as igfile:
        lines = igfile.readlines()
        for line in lines:
            line = line.split(",")
            frame = int(line[0])
            x = int(line[2])
            y = int(line[3])
            w = int(line[4])
            h = int(line[5])
            classe = int(line[-1])
            if frame in frame_boxes:
                frame_boxes[frame] += [[frame, x, y, w, h, classe]]
            else:
                frame_boxes[frame] = [[frame, x, y, w, h, classe]]

    # Para cada vídeo, ajustar as imagens do UAVDT Local com os devidos blurs..
    for frame, boxes in frame_boxes.items():
        img_path = f"./UAVDT_YOLOv5/train/images/{movie_name}_img{frame:06}.jpg"
        img = cv2.imread(img_path)
        for _,x,y,w,h,_ in boxes:
            # Kernel Sized
            kw = w//2
            kh = h//2
            if kw % 2 == 0: kw -= 1
            if kh % 2 == 0: kh -= 1
            # Gaussian Blur
            roi = img[y:y+h, x:x+w]
            roi_blur = cv2.GaussianBlur(roi, (kw, kh), 20, 20, cv2.BORDER_REPLICATE)
            img[y:y+h, x:x+w] = roi_blur
        
        cv2.imwrite(img_path, img)

"""
# Checar efeitos do Gaussian Blur em uma "ignore area" de uma imagem qualquer do dataset.

img = cv2.imread("../../Datasets/UAVDT/UAV-benchmark-M/M0101/img000001.jpg")
# Boxes da imagem 000001.jpg do filme M0101; [frame, x, y, w, h, classe]
frame1_boxes_img000001_M0101 = [[1, 914, 189, 105, 54, -1],  [1, 956, 480, 64, 58, -1],  [1, 788, 291, 121, 113, -1]]

for _,x,y,w,h,_ in frame1_boxes_img000001_M0101:
    # Kernel Sized
    kw = w//2
    kh = h//2
    if kw % 2 == 0: kw -= 1
    if kh % 2 == 0: kh -= 1
    # 2xGaussian Blur
    roi = img[y:y+h, x:x+w]
    roi_blur = cv2.GaussianBlur(roi, (kw, kh), 20, 20, cv2.BORDER_REPLICATE)
    img[y:y+h, x:x+w] = roi_blur

cv2.imshow("Teste", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""
pass

# Criar o arquivo data.yaml

In [ ]:
# O conteúdo original não está adequado... mais fácil que editar: removerei-o e criarei um novo.
%cd {root}

datayaml = """# train and val datasets (image directory or *.txt file with image paths)

train: ../UAVDT_YOLOv5/train/images/
val: ../UAVDT_YOLOv5/test/images/

# number of classes
nc: 3

# class names
names: ['car', 'truck', 'bus']
"""

%rm data.yaml
with open('data.yaml', 'w') as file:
    file.write(datayaml)
%cat data.yaml

# Iniciar o Treinamento por 300 eṕocas


In [ ]:
%cd {root}/yolov5
epocas = 300

!python train.py --epochs {epocas} --cfg ./models/yolov5l.yaml --weights ../../../Weights/YoloV5/yolov5l.pt  --data ../data.yaml --img 640 --batch-size 16 --device 1 --logdir ../runs --workers 4


# Guardar os Resultados na Nuvem (Dropbox)

In [ ]:
%cd {root}
%mkdir -p {output_dir}
%cp -av logs/ {output_dir}
%cp -av runs/ {output_dir}

# Finalização

In [ ]:
finish_time = datetime.datetime.now()
delta_time = finish_time - start_time
print("""
Start: {start}

Finish: {finish}

Duration (hour:min:sec.microsec):  {delta}
""".format(start = start_time, finish = finish_time, delta = delta_time))

In [ ]:
# Manualmente, remover os logs e diretórios com imagens desnecessários.